In [23]:
import pandas as pd
import configparser
import os
import xlrd

In [24]:
config = configparser.RawConfigParser()
config.read('/Users/zachblumenfeld/devtools/msom-demo.ini')
HOST = config['NEO4J']['HOST']
USERNAME = config['NEO4J']['USERNAME']
PASSWORD = config['NEO4J']['PASSWORD']

In [25]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=True)

In [26]:
# get source data from:
# https://pubsonline.informs.org/doi/suppl/10.1287/msom.1070.0176/suppl_file/msom.1070.0176-sm-datainexcel.zip
xls = pd.ExcelFile('./data/MSOM-06-038-R2 Data Set in Excel.xls')

In [27]:
arc_dfs = []
stage_dfs = []
for sheet_name in xls.sheet_names:
    if sheet_name[-2:] == 'LL':
        df = xls.parse(sheet_name)
        df['chainId'] = int(sheet_name[:2])
        df['sourceStageId'] = sheet_name[:2] + '_' + df['sourceStage']
        df['destinationStageId'] = sheet_name[:2] + '_' + df['destinationStage']
        arc_dfs.append(df)
    elif sheet_name[-2:] == 'SD':
        df = xls.parse(sheet_name)
        df['chainId'] = int(sheet_name[:2])
        df['stageId'] = sheet_name[:2] + '_' + df['Stage Name']
        stage_dfs.append(df)


In [28]:
arc_df = pd.concat(arc_dfs)
arc_df

,sourceStage,destinationStage,chainId,sourceStageId,destinationStageId
0,Manuf_0001,Retail_0001,1,01_Manuf_0001,01_Retail_0001
1,Manuf_0001,Retail_0002,1,01_Manuf_0001,01_Retail_0002
2,Manuf_0002,Retail_0002,1,01_Manuf_0002,01_Retail_0002
3,Manuf_0002,Retail_0003,1,01_Manuf_0002,01_Retail_0003
4,Part_0001,Manuf_0001,1,01_Part_0001,01_Manuf_0001
...,...,...,...,...,...
16220,Trans_0555,Retail_0555,38,38_Trans_0555,38_Retail_0555
16221,Trans_0556,Retail_0556,38,38_Trans_0556,38_Retail_0556
16222,Trans_0557,Retail_0557,38,38_Trans_0557,38_Retail_0557
16223,Trans_0558,Retail_0558,38,38_Trans_0558,38_Retail_0558


In [29]:
stage_df = pd.concat(stage_dfs)
rename_st_list = [i for i in stage_df.columns if 'stageTime_' in i]
unnamed_list = [i for i in stage_df.columns if 'Unnamed' in i]
stage_df = stage_df.drop(columns=unnamed_list)\
    .rename(columns={'Stage Name':'stageName', 'stdDev stageTime':'stdDevStageTime'})\
    .rename(columns=dict(zip(rename_st_list,[i.replace('_','').replace('%','Perc') for i in rename_st_list])))

stage_df

,stageName,stageCost,relDepth,stageClassification,avgDemand,stdDevDemand,maxServiceTime,serviceLevel,stageTime,stdDevStageTime,...,stageTime4,stageTimePerc4,stageTime5,stageTimePerc5,stageTime6,stageTimePerc6,xPosition,yPosition,chainId,stageId
0,Manuf_0001,39.00,1,Manuf,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,176,32,1,01_Manuf_0001
1,Manuf_0002,36.00,1,Manuf,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,176,96,1,01_Manuf_0002
2,Part_0001,12.00,2,Part,NaN,NaN,NaN,NaN,28.0,11.224972,...,NaN,NaN,NaN,NaN,NaN,NaN,32,32,1,01_Part_0001
3,Part_0002,5.00,2,Part,NaN,NaN,NaN,NaN,15.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,32,96,1,01_Part_0002
4,Part_0003,9.00,2,Part,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,32,160,1,01_Part_0003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,Trans_0555,0.44,1,Trans,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,528,17760,38,38_Trans_0555
2021,Trans_0556,0.45,1,Trans,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,528,17792,38,38_Trans_0556
2022,Trans_0557,0.38,1,Trans,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,528,17824,38,38_Trans_0557
2023,Trans_0558,0.38,1,Trans,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,528,17856,38,38_Trans_0558


In [31]:
## Correct Data Entry Inconsistency. 'Parts' in chain 19  is that same as 'Part' in other chains
stage_df.loc[stage_df.stageClassification == 'Parts' ,'stageClassification'] = 'Part'

In [32]:
gds.run_cypher('CREATE CONSTRAINT stage_unique IF NOT EXISTS ON (n:Stage) ASSERT n.stageId IS UNIQUE')

""


In [34]:
# load nodes
classifications = stage_df.stageClassification.unique()
for i in classifications:
    records = stage_df.loc[stage_df.stageClassification == i,
            ['stageName', 'stageCost', 'relDepth', 'stageTime', 'chainId', 'stageId']].to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $records AS rec
        MERGE(s:Stage {{stageId:rec.stageId}})
        ON CREATE SET s:{i},
            s.stageName = rec.stageName,
            s.stageCost = rec.stageCost,
            s.relDepth = rec.relDepth,
            s.stageTime = rec.stageTime,
            s.chainId = rec.chainId
        RETURN labels(s), count(s)
    ''', params={'records':records})
    print(res)

        labels(s)  count(s)
0  [Stage, Manuf]      3214
       labels(s)  count(s)
0  [Stage, Part]      5768
         labels(s)  count(s)
0  [Stage, Retail]      3553
        labels(s)  count(s)
0  [Stage, Trans]      1913
       labels(s)  count(s)
0  [Stage, Dist]      1424


In [35]:
# load relationships
records = arc_df[['sourceStageId','destinationStageId']].to_dict('records')
gds.run_cypher('''
        UNWIND $records AS rec
        MATCH(src:Stage {stageId:rec.sourceStageId})
        MATCH(dst:Stage {stageId:rec.destinationStageId})
        MERGE(src)-[r:SUPPLIES]->(dst)
        RETURN count(r)
    ''', params={'records':records})

,count(r)
0,43589
